In [2]:
import pandas as pd
import numpy as np
import requests
from tqdm import tqdm

In [3]:
brand_df = pd.read_csv('db_cosmenet/brand_name.csv')
category_df = pd.read_csv('db_cosmenet/category.csv')
product_df = pd.read_csv('db_cosmenet/product_joined.csv')
scan_df = pd.read_csv('/app/nfs_clientshare/Datasets/Cosmenet_uat_20231108/data_last_join_2023_12_18.csv')

In [4]:
brand_df_clean = brand_df[brand_df['DEPTH_LEVEL'] == 2].copy()
brand_df_clean.drop(['DEPTH_LEVEL', 'IBLOCK_SECTION_ID'], axis=1, inplace=True)
brand_df_clean.reset_index(drop=True, inplace=True)
brand_df_clean.tail(1)

,ID,NAME
1350,4681,CHAT


In [5]:
category_df_clean = category_df[category_df['DEPTH_LEVEL'] == 1].reset_index(drop=True)
category_df_clean.drop(['DEPTH_LEVEL', 'IBLOCK_SECTION_ID'], axis=1, inplace=True)
category_df_clean.tail(1)

,ID,NAME
9,1821,For Men


In [6]:
subcategory_df_clean = category_df[category_df['DEPTH_LEVEL'] == 3].reset_index(drop=True)[['ID', 'IBLOCK_SECTION_ID', 'NAME']]
subcategory_df_clean = pd.merge(subcategory_df_clean, category_df[['ID', 'IBLOCK_SECTION_ID']], left_on='IBLOCK_SECTION_ID', right_on='ID', how='left')
subcategory_df_clean = pd.merge(subcategory_df_clean, category_df[['ID']], left_on='IBLOCK_SECTION_ID_y', right_on='ID', how='left')
subcategory_df_clean.drop(['IBLOCK_SECTION_ID_x', 'ID_y', 'IBLOCK_SECTION_ID_y'], axis=1, inplace=True)
subcategory_df_clean.columns = ['SCID', 'NAME', 'CID']

subcategory_df_not_parent = subcategory_df_clean[subcategory_df_clean['CID'].isna()]
subcategory_df_clean.drop(subcategory_df_not_parent.index, inplace=True)
subcategory_df_clean.reset_index(drop=True, inplace=True)
subcategory_df_clean.tail(1)

,SCID,NAME,CID
130,2683,Home Perfume,63


In [7]:
product_df_BID_not_provided = product_df[product_df['BID'] < 0]
product_df_clean = product_df.drop(product_df_BID_not_provided.index)
product_df_BID_not_have = product_df_clean[product_df_clean['BID'].isin(brand_df_clean['ID']) == False]
product_df_clean.drop(product_df_BID_not_have.index, inplace=True)

product_df_CID_not_provided = product_df_clean[product_df_clean['CID'] < 0]
product_df_clean = product_df_clean.drop(product_df_CID_not_provided.index)
product_df_CID_not_have = product_df_clean[product_df_clean['CID'].isin(category_df_clean['ID']) == False]
product_df_clean.drop(product_df_CID_not_have.index, inplace=True)

product_df_SCID_not_provided = product_df_clean[product_df_clean['SCID'] < 0]
product_df_clean = product_df_clean.drop(product_df_SCID_not_provided.index)
product_df_SCID_not_have = product_df_clean[product_df_clean['SCID'].isin(subcategory_df_clean['SCID']) == False]
product_df_clean.drop(product_df_SCID_not_have.index, inplace=True)

product_df_clean.reset_index(drop=True, inplace=True)
product_df_clean.tail(1)

,EID,PRODUCT_NAME,BID,BRAND_NAME,SCID,SUBCAT_NAME,CID,CAT_NAME
29901,49169,Bio Booster Skin Brightening Sun Surum SPF 50 ...,3992,Am Herb,248,Facial Sunscreen,62,Sun Product


In [8]:
scan_df_EID_not_have = scan_df[scan_df['labels'].isin(product_df_clean['EID']) == False]
scan_df_clean = scan_df.drop(scan_df_EID_not_have.index)
scan_df_clean.reset_index(drop=True, inplace=True)
scan_df_clean.tail(1)

,file_names,labels,images_path,BID,SCID,CID
93613,Be-You-Tiful-Eyeshadow-Palette.jpg,37992,/app/nfs_clientshare/Datasets/Cosmenet_uat_202...,2069,227,58


In [9]:
def insert_data(body, tag_name):
    response = requests.post(
        url="http://127.0.0.1:8000/" + tag_name + "/insert", 
        json=body,
        headers={"Content-Type": "application/json"},
    )
    if response.status_code != 201:
        print(response.status_code)
        print(response.text)
        return False
    return True

In [20]:
brand_body = {
  "active": 'true',
  "brand_id": "",
  "brand_name": "",
  "description": "",
  "update_by": "admin",
  "url": "https://www.google.com/",
  "url_preview_image": "https://www.google.com/"
}

for i in tqdm(range(len(brand_df_clean))):
    brand_body['brand_id'] = str(brand_df_clean['ID'][i])
    brand_body['brand_name'] = brand_df_clean['NAME'][i]
    if not insert_data(brand_body, 'brand'):
        print(f'index {i}')

100%|██████████| 1351/1351 [00:25<00:00, 53.24it/s]


In [22]:
category_body = {
  "active": 'true',
  "category_id": "",
  "category_name": "",
  "description": "test",
  "update_by": "admin",
  "url": "https://www.google.com/",
  "url_preview_image": "https://www.google.com/"
}

for i in tqdm(range(len(category_df_clean))):
    category_body['category_id'] = str(category_df_clean['ID'][i])
    category_body['category_name'] = category_df_clean['NAME'][i]
    if not insert_data(category_body, 'category'):
        print(f'index {i}')

100%|██████████| 10/10 [00:00<00:00, 34.33it/s]


In [24]:
subcategory_body = {
  "active": 'true',
  "category_id": "",
  "description": "test",
  "sub_category_id": "",
  "sub_category_name": "",
  "update_by": "admin",
  "url": "https://www.google.com/",
  "url_preview_image": "https://www.google.com/"
}

for i in tqdm(range(len(subcategory_df_clean))):
    subcategory_body['category_id'] = str(subcategory_df_clean['CID'][i])
    subcategory_body['sub_category_id'] = str(subcategory_df_clean['SCID'][i])
    subcategory_body['sub_category_name'] = subcategory_df_clean['NAME'][i]
    if not insert_data(subcategory_body, 'subcategory'):
        print(f'index {i}')

100%|██████████| 131/131 [00:02<00:00, 46.98it/s]


In [37]:
product_body = {
  "active": 'true',
  "brand_id": "",
  "description": "test",
  "product_id": "",
  "product_name": "",
  "sub_category_id": "",
  "update_by": "admin",
  "url": "https://www.google.com/",
  "url_preview_image": "https://www.google.com/"
}

for i in tqdm(range(len(product_df_clean))):
    product_body['brand_id'] = str(product_df_clean['BID'][i])
    product_body['product_id'] = str(product_df_clean['EID'][i])
    product_body['product_name'] = product_df_clean['PRODUCT_NAME'][i]
    product_body['sub_category_id'] = str(product_df_clean['SCID'][i])
    if not insert_data(product_body, 'product'):
        print(f'index {i}')

100%|██████████| 70/70 [00:03<00:00, 23.28it/s]


In [ ]:
scan_body = {
  "active": 'true',
  "counter": '0',
  "directory_name": "test",
  "image_path": "",
  "product_id": "",
  "update_by": "admin"
}

for i in tqdm(range(len(scan_df_clean))):
    scan_body['image_path'] = scan_df_clean['images_path'][i]
    scan_body['product_id'] = str(scan_df_clean['labels'][i])
    if not insert_data(scan_body, 'main'):
        print(f'index {i}')

In [13]:
scan_df_clean.iloc[0]['images_path']

'/app/nfs_clientshare/Datasets/Cosmenet_product_20231018/datas/14624/14624_14.jpg'

In [14]:
pred_body = {
  "brand_id": "",
  "category_id": "",
  "image_path": "/app/nfs_clientshare/Datasets/Cosmenet_product_20231018/datas/14624/14624_14.jpg",
  "page": 1,
  "size": 5,
  "sub_category_id": "",
  "update_by": ""
}

In [122]:
response = requests.post(
    url="http://127.0.0.1:8000/" + 'predict' + "/product", 
    json=pred_body,
    headers={"Content-Type": "application/json"},
)